### Challenge 17.
### Validate Binary Search Tree.
### *[LeetCode 98.](https://leetcode.com/problems/validate-binary-search-tree/)*

**Description**

Given the root of a binary tree, determine if it is a valid binary search tree (BST). 

A valid BST is defined as follows:
- The left subtree of a node contains only nodes with keys less than the node's key.
- The right subtree of a node contains only nodes with keys greater than the node's key.
- Both the left and right subtrees must also be binary search trees.

**Examples**

*Example 1*\
Input: root = [2,1,3]\
Output: true

*Example 2*\
Input: root = [5,1,4,null,null,3,6]\
Output: false\
Explanation: The root node's value is 5 but its right child's value is 4.
 
**Constraints**

The number of nodes in the tree is in the range [1, 10^4].\
-2^31 <= Node.val <= 2^31 - 1

### Proposed Solution

In [4]:
### Challenge 17
### Validate Binary Search Tree
### Proposed Solution



### Definition of auxiliary classes

# Class for a tree node
class Node:
    
    # Definition of Node
    def __init__(self, data, child_l=None, child_r=None):
        self.data = data
        self.child_l = child_l
        self.child_r = child_r

    # Representation of Node
    def __repr__(self):
        rep = "%s -> (%s, %s)" % (str(self.data), str(self.child_l), str(self.child_r))
        return rep


# Class for a Tree
class Tree:

    # Definition of Tree
    def __init__(self, nodes=None|list):

        # Trivial definition
        self.root = Node(None)
        self.deps = None
        self.pars = None
        self.hist = None
        self.nodes = None

        #### Definition from list of nodes
        if (nodes is not None):

            ### Tree root
            self.root = Node(nodes[0])

            ### Identify nodes properties
            # dep: node depth (1,2,3,...)
            # dir: node direction from parent perspective, left ("l") or right ("r")
            # par: indicator of parent location (index + 1)
            # his: consecutive directions from root to node
            ls_deps, ls_dirs, ls_pars, ls_hist = [1], ["-"], [0], [["-"]]

            ### Find properties of each node
            for aux_count in range(2,len(nodes)+1):
                
                ## Find dep, dir, par
                dep = len(bin(aux_count))-2
                dir = "l" if aux_count%2 == 0 else "r"
                par = int(aux_count/2)
                ls_deps.append(dep)
                ls_dirs.append(dir)
                ls_pars.append(par)

                ## Find his                
                # Instantiate parent node location (par) and node history (his) before iterating to the root
                idx = aux_count-1
                par = ls_pars[idx]
                his = []
                # Iterate over parent nodes until reaching the root
                while par != 0:
                    # Add dir from previous parent node to his 
                    dir = ls_dirs[idx]
                    his = [dir] + his
                    # Recalculate previous parent node location for next iteration
                    idx = par-1
                    par = ls_pars[idx]
                # Save his
                ls_hist.append(his)

            ### Build tree from nodes properties
                
            ## Identify how many parent nodes are
            max_dep = max(ls_deps)
            range_parnts = len([elem for elem in ls_deps if elem < max_dep])

            ## Build tree by iterating over all parent nodes
            for ii in range(range_parnts):

                # Retrieve children properties
                # ls_chs: list of children nodes
                # ls_his: list of children history (consecutive directions from root to node)
                par_id = ii+1
                ls_chs = [nodes[ii] for ii in range(len(nodes)) if ls_pars[ii] == par_id]
                ls_his = [ls_hist[ii] for ii in range(len(nodes)) if ls_pars[ii] == par_id] 

                # Assign children nodes
                if (len(ls_chs) == 2):
                    self.add_child(node=Node(ls_chs[0]), directions=ls_his[0], init=True)
                    self.add_child(node=Node(ls_chs[1]), directions=ls_his[1], init=True)
                if len(ls_chs) == 1:
                    self.add_child(node=Node(ls_chs[0]), directions=ls_his[0], init=True)

            ### Save nodes properties
            self.deps = ls_deps
            self.pars = ls_pars
            self.hist = ls_hist
            self.nodes = nodes


    # Representation of Tree
    def __repr__(self):       
        return self.root.__repr__()


    # Move through Tree starting from the root 
    def move(self, directions:list):
        # Starting point is the root
        t_aux = self.root
        for dir in directions:
            # Move to left
            if dir == "l":
                t_aux = t_aux.child_l
            # Move to right
            elif dir == "r":
                t_aux = t_aux.child_r
            # Do not move
            else:
                t_aux = t_aux
        return t_aux


    # Add child node to Tree
    def add_child(self, node:Node, directions:list, init:bool=False):

        # Locate parent node
        t_aux = self.move(directions[:-1])
        # Direction of child node
        dir = directions[-1]
        # Adding child node to parent node
        if node.data is not None:
            if dir == "l":
                t_aux.child_l = node
            else:
                t_aux.child_r = node

        # Setting node properties when a single node is added (not via __init__)
        if init == False:

            # Reproduce aux_count from directions
            aux_count = 1
            power = 1
            for dir in directions[::-1]:
                if dir == "l":
                    aux_count += 1**(power)
                elif dir == "r":
                    aux_count += 2**(power)
                else:
                    aux_count += 0
                power += 1

            # Identifying position of new nodes based on a masking method
            ln = len(self.nodes)
            len_nodes = (2**power)-1
            mask_nodes = [1 if ii+1 <= ln else 0 for ii in range(len_nodes)]
            mask_nodes[aux_count-1] = 2
            
            # Populating deps, pars, hist, and nodes
            nodes, deps, pars, hist = [], [], [], []
            for ii in range(len_nodes):
                aux_count = ii + 1
                # Masking = 1 -> Node already exists in the Tree
                if mask_nodes[ii] == 1:
                    deps.append(self.deps[ii])
                    pars.append(self.pars[ii])
                    hist.append(self.hist[ii])
                    nodes.append(self.nodes[ii])
                # Masking = 2 -> Node recently added to the Tree via add_child
                elif mask_nodes[ii] == 2:
                    deps.append(len(bin(aux_count))-2)
                    pars.append(int(aux_count/2))
                    hist.append(directions)
                    nodes.append(node.data)
                # Masking = 0 -> None node must be added to preserve lists length of deps, pars, hist, and nodes
                else:
                    deps.append(len(bin(aux_count))-2)
                    pars.append(int(aux_count/2))
                    hist.append([*str(bin(aux_count))[3:].replace("0","l").replace("1","r")])
                    nodes.append(None)

            # Update deps, pars, hist, and nodes as attributes of Tree
            self.deps = deps
            self.pars = pars
            self.hist = hist
            self.nodes = nodes


    # Traverse Tree (pre-order traversal)
    def traverse(self, node:Node):
        ret = []
        if node:
            ret.append(node.data)
            ret = ret + self.traverse(node.child_l)
            ret = ret + self.traverse(node.child_r)
        return ret



### Definition of the class solution

class Solution17:

    # Validate input
    def check_input(self, root:list) -> bool:

        # Initialize validations
        # nn: the tree must have between 1 and 10^4 nodes
        # nv: nodes values should be between -2^31 and 2^31 - 1
        nn, nv = True, True

        # Perform validations
        clean_root = [elem for elem in root if elem != None]
        if (len(clean_root) < 1) or (len(clean_root) > 1e4):
            nn = False
        for elem in clean_root:
            if (float(elem) < -2e31) or (float(elem) > 2e31):
                nv = False

        return all([nn,nv])


    # Binary search tree validation by node
    def bst_validation(self, root:list) -> dict:

        # Create a Tree from input
        t = Tree(nodes=root)

        # Identify parent nodes
        max_dep = max(t.deps)
        range_parnts = len([elem for elem in t.deps if elem < max_dep])

        # Initialize variables where result will be stored
        dc_validations = dict()
        valid_bst = True
        count = 0

        # Iterate over all parent nodes
        for his in t.hist[0:range_parnts]:

            # Locate on current parent node
            t_aux = t.move(his)

            # Left sub-branch validation: verify if left sub-branch of current node is a BST
            lsbv = True
            if (t_aux is not None) and (t_aux.child_l is not None):
                if t_aux.child_l.data < t_aux.data:
                    children = t.traverse(node=t_aux.child_l)
                    if children != []:
                        children_validation = [ch for ch in children if (t_aux.data - ch) > 0]
                        if len(children_validation) != len(children):
                            lsbv = False
                else:
                    lsbv = False

            # Right sub-branch validation: verify if right sub-branch of current node is a BST
            rsbv = True
            if (t_aux is not None) and (t_aux.child_r is not None):
                if t_aux.child_r.data > t_aux.data:
                    children = t.traverse(node=t_aux.child_r)
                    if children != []:
                        children_validation = [ch for ch in children if (t_aux.data - ch) < 0]
                        if len(children_validation) != len(children):
                            rsbv = False
                else:
                    rsbv = False
            
            # Store results by node
            if t_aux is None: count += 1
            key = t_aux.data if t_aux is not None  else "NullNode"+str(count)
            value = [lsbv, rsbv]
            dc_validations[key] = value

        # Store overall results
        for value in dc_validations.values():
            if all(value) == False:
                valid_bst = False
        dc_validations["result"] = valid_bst

        return dc_validations


    # Binary search tree validation (main function)
    def main(self, root:list) -> dict:

        # Validation of inputs
        input_valid = self.check_input(self, root)

        # Case when input is valid
        if input_valid:

            # Retrieve the result
            dc_validations = self.bst_validation(self, root)
            dc_result = {
                "input_valid": input_valid,
                "bst_as_list": root,
                "bst_as_tree": Tree(nodes=root),
                "dc_validations": dc_validations,
                "bst_valid": dc_validations["result"]
            }

        # Case when input is not valid
        else:
            # Retrieve the result
            dc_result = {
                "input_valid": input_valid,
                "bst_as_list": root
            }

        return dc_result

In [5]:
### Testing Proposed Solution
### Execution of the main function

# Initialize inputs
root1 = [2,1,3]
root2 = [5,1,4,None,None,3,6]
root3 = []
root4 = [8,4,12,2,6,10,14,1,3,5,7,9,11,13,15]
root5 = [9,6,11,4,None,8,10,None,None,None,None,5]

# Executions
print("------------ Example 1 ------------")
dc_result_1 = Solution17.main(Solution17, root1)
print(dc_result_1)
print("------------ Example 2 ------------")
dc_result_2 = Solution17.main(Solution17, root2)
print(dc_result_2)
print("------------ Example 3 ------------")
dc_result_3 = Solution17.main(Solution17, root3)
print(dc_result_3)
print("------------ Example 4 ------------")
dc_result_4 = Solution17.main(Solution17, root4)
print(dc_result_4)
print("------------ Example 5 ------------")
dc_result_5 = Solution17.main(Solution17, root5)
print(dc_result_5)

------------ Example 1 ------------
{'input_valid': True, 'bst_as_list': [2, 1, 3], 'bst_as_tree': 2 -> (1 -> (None, None), 3 -> (None, None)), 'dc_validations': {2: [True, True], 'result': True}, 'bst_valid': True}
------------ Example 2 ------------
{'input_valid': True, 'bst_as_list': [5, 1, 4, None, None, 3, 6], 'bst_as_tree': 5 -> (1 -> (None, None), 4 -> (3 -> (None, None), 6 -> (None, None))), 'dc_validations': {5: [True, False], 1: [True, True], 4: [True, True], 'result': False}, 'bst_valid': False}
------------ Example 3 ------------
{'input_valid': False, 'bst_as_list': []}
------------ Example 4 ------------
{'input_valid': True, 'bst_as_list': [8, 4, 12, 2, 6, 10, 14, 1, 3, 5, 7, 9, 11, 13, 15], 'bst_as_tree': 8 -> (4 -> (2 -> (1 -> (None, None), 3 -> (None, None)), 6 -> (5 -> (None, None), 7 -> (None, None))), 12 -> (10 -> (9 -> (None, None), 11 -> (None, None)), 14 -> (13 -> (None, None), 15 -> (None, None)))), 'dc_validations': {8: [True, True], 4: [True, True], 12: [Tru

### [External Solution](https://leetcode.com/problems/validate-binary-search-tree/solutions/3817712/best-fastest-solution-easy-to-understand-for-beginners-python/)

In [7]:
### Challenge 17
### Validate Binary Search Tree
### External Solution



### Definition of auxiliary classes

# Class for the tree node
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right


### Definition of the solution

# Class for the solution
class Solution:    
    
    # Start recursion
    def isValidBST(self, root_node:TreeNode) -> bool:
        
        # Start the recursive checking with initial minimum and maximum values as -infinity and +infinity
        return self.is_valid_bst_helper(self, root_node, float('-inf'), float('inf'))

    
    # Recursive calls
    def is_valid_bst_helper(self, node, min_val, max_val):

        if not node:
            return True
            
        # Check if the current node's value is within the valid range
        if not (min_val < node.val < max_val):
            return False
            
        # Check left subtree with updated max_val
        left_valid = self.is_valid_bst_helper(self, node.left, min_val, node.val)
        # Check right subtree with updated min_val
        right_valid = self.is_valid_bst_helper(self, node.right, node.val, max_val)
        
        # Both subtrees must be valid BSTs for the whole tree to be a valid BST
        return left_valid and right_valid


    # Function for i/o formatting     
    def io_format(self, root:list):
        
        # Formatting input (from list to TreeNode)
        if len(root) > 0:
            root_node = TreeNode(val=root[0])
            nodes = [root_node]
            for i, x in enumerate(root[1:]):
                if x is None:
                    continue
                parent_node = nodes[i // 2]
                is_left = (i % 2 == 0)
                node = TreeNode(val=x)
                if is_left:
                    parent_node.left = node
                else:
                    parent_node.right = node
                nodes.append(node)
        else:
            return TreeNode(val=None)
    
        # Apply isValidBST function
        result = self.isValidBST(self, root_node)
        
        # Format output
        return result

In [8]:
### Testing External Solution

# Initialize inputs
root1 = [2,1,3]
root2 = [5,1,4,None,None,3,6]
root3 = []
root4 = [8,4,12,2,6,10,14,1,3,5,7,9,11,13,15]
root5 = [9,6,11,4,None,8,10,None,None,None,None,5]

# Executions
print("------------ Example 1 ------------")
result_1 = Solution.io_format(Solution, root1)
print(result_1)
print("------------ Example 2 ------------")
result_2 = Solution.io_format(Solution, root2)
print(result_2)
print("------------ Example 3 ------------")
result_3 = Solution.io_format(Solution, root3)
print(result_3)
print("------------ Example 4 ------------")
result_4 = Solution.io_format(Solution, root4)
print(result_4)
print("------------ Example 5 ------------")
result_5 = Solution.io_format(Solution, root5)
print(result_5)

------------ Example 1 ------------
True
------------ Example 2 ------------
False
------------ Example 3 ------------
------------ Example 4 ------------
True
------------ Example 5 ------------
False


### Final Remarks

This challenge is a typical exercise about [Trees](https://www.geeksforgeeks.org/introduction-to-tree-data-structure-and-algorithm-tutorials/?ref=shm). Let me recap the thought process behind the proposed solution.

First of all, the tree structure was defined using two classes `Node` and `Tree`. The`Node` class sets attributes for each node (value, left child, and right child), while the `Tree` class defines operations that can be applied to nodes. The methods of the `Tree` class are:

1. __\__init__\__: a Tree object should be initialized with a non-empty list. The Tree object has the following attributes.\
    1.1. _self.root_: node that represents the tree root.\
    1.2. _self.nodes_: list of node values.\
    1.3. _self.deps_: depth of each node (a.k.a. level) is given. The lowest value is 1 which corresponds to the tree root.\
    1.4. _self.pars_: location (index + 1) of each parent node is given.\
    1.5. _self.hist_: history of consecutive binary moves (left or right) from root to each node.
2. __\__repr__\__: string representation of a Tree object.
3. _move_: allows to go from the root towards any node, using a sequence of single movements (left or right steps).
4. _add_child_: add a child node to a specific node.
5. _traverse_: traverse the Tree object following the root-left-right direction. This algorithm is known as pre-order traversal.

All the aforementioned methods were developed in-house, except by _traverse_ which was taken from [this external source](https://www.educative.io/blog/essential-tree-traversal-algorithms). 

Regarding class `Solution17`, its core is just another method for the class `Tree` called _bst_validation_. The latter was devised to validate if the input list _root_ recreates a valid [Binary Search Tree (BST)](https://www.programiz.com/dsa/binary-search-tree). This is how it works. For each node it is evaluated if its children from the left subtree are less than node value, and if its children from the right subtree are bigger than the node value. The process is repeated for all nodes, from root to deeper nodes, thanks to the _traverse_ method.

On the other hand, the external solution works barely in the same way as the proposed solution. There two main differences: the definition of trees and the execution of the BST validation. Trees are defined as sequences of nested nodes of class `TreeNode`, which does not have any additional methods beyond __\__init__\__. For creating a tree from a list it was used a piece of code from [this source](https://stackoverflow.com/questions/71216876/build-a-binary-tree-from-a-python-list), that appears in the method _io_format_ from class `Solution`. Besides, the BST validation was done recursively without the need of a traverse algorithm. 

There are so many things to learn from this exercise. Every single step represented a challenge itself. The key learnings are summarized here:
- It took a lot of time to develop the `Tree` class. This was based on the mathematical properties of a binary tree, but not in the computational advantages of the data structure itself. That is why it was decided to copy an efficient traverse method from an external source, instead of developing it from scratch.
- The comparison between the lines of code of both solutions says a lot about the understanding and efficient usage of the tree data structure.
- Applying recursive methods is an elegant approach to solve problems like this.
- The proposed solution gives as result a Tree object with many properties that allow to understand better the dependencies between nodes.

Additional notes can be found below to deepen the discussion about binary trees.

### [Binary Trees](https://www.educative.io/answers/binary-trees-in-python), their [traverse methods](https://www.educative.io/blog/essential-tree-traversal-algorithms), and [deletion of nodes](https://www.youtube.com/watch?v=LFzAoJJt92M)

In [12]:
### Auxiliary notes about Binary Trees
### Definition of a Binary Tree

# Class for a BST Tree
class Tree:
    
    # Initialize object with a single node
    def __init__(self, data):
        self.data = data
        self.leftChild = None
        self.rightChild = None

    # Method to insert a node in BST
    def insert(self, data):
        # if value is lesser than the value of the parent node
        if data < self.data:
            if self.leftChild:
                # if we still need to move towards the left subtree
                self.leftChild.insert(data)
            else:
                self.leftChild = Tree(data)
                return
        # if value is greater than the value of the parent node        
        else:
            if self.rightChild:
                # if we still need to move towards the right subtree
                self.rightChild.insert(data)
            else:
                self.rightChild = Tree(data)
                return
            
    # Method to search a node in BST
    def search(self, val):
        # if value to be searched is found
        if val==self.data:
            return str(val)+" is found in the BST"
        # if value is lesser than the value of the parent node
        elif val < self.data:
            # if we still need to move towards the left subtree
            if self.leftChild:
                return self.leftChild.search(val)
            else:
                return str(val)+" is not found in the BST"
        # if value is greater than the value of the parent node
        else:
            # if we still need to move towards the right subtree
            if self.rightChild:
                return self.rightChild.search(val)
            else:
                return str(val)+" is not found in the BST" 

    # Method to delete a node in BST
    def delete(self, root, key:int):
        if not root:
            return root
        
        if key > root.data:
            root.rightChild = self.delete(root.rightChild, key)
        elif key < root.data:
            root.leftChild = self.delete(root.leftChild, key)
        else:
            if not root.leftChild:
                return root.rightChild
            elif not root.rightChild:
                return root.leftChild
            
            # Find the min form right subtree
            cur = root.rightChild
            while cur.leftChild:
                cur = cur.leftChild
            root.data = cur.data
            root.rightChild = self.delete(root.rightChild, root.data)
        return root
            
    # Method for in-order traversal
    # leftChild -> parent -> rightChild
    def inorderTraversal(self, root):
        ret = []
        if root:
            ret = self.inorderTraversal(root.leftChild)
            ret.append(root.data)
            ret = ret + self.inorderTraversal(root.rightChild)
        return ret            

    # Method for pre-order traversal
    # parent -> leftChild -> rightChild
    def preorderTraversal(self, root):
        ret = []
        if root:
            ret.append(root.data)
            ret = ret + self.preorderTraversal(root.leftChild)
            ret = ret + self.preorderTraversal(root.rightChild)
        return ret        

    # Method for post-order traversal
    # leftChild -> rightChild -> parent
    def postorderTraversal(self, root):
        ret = []
        if root:
            ret = ret + self.postorderTraversal(root.leftChild)
            ret = ret + self.postorderTraversal(root.rightChild)
            ret.append(root.data)
        return ret   
    
    # Method to print a BST
    def __repr__(self):
        rep = "%s -> (%s, %s)" % (str(self.data), str(self.leftChild), str(self.rightChild))
        return rep
    
# Initialize object with multiple nodes
def tree_from_list(elements):
    root = Tree(data=elements[0])
    nodes = [root]
    for i, x in enumerate(elements[1:]):
        if x is None:
            continue
        parent_node = nodes[i // 2]
        is_left = (i % 2 == 0)
        node = Tree(data=x)
        if is_left:
            parent_node.leftChild = node
        else:
            parent_node.rightChild = node
        nodes.append(node)
    return root

In [13]:
### Tree class on action
### Examples of Tree (__init__ and insert)

# Creating root node
root = Tree(5)
print("Initialized Tree")
print(root)
print("")

# Inserting values in BST
root.insert(3)
root.insert(7)
root.insert(2)
root.insert(4)
root.insert(6)
root.insert(8)
print("Inserting new nodes to the Tree")
print(root)

Initialized Tree
5 -> (None, None)

Inserting new nodes to the Tree
5 -> (3 -> (2 -> (None, None), 4 -> (None, None)), 7 -> (6 -> (None, None), 8 -> (None, None)))


In [14]:
### Tree class on action
### Examples of Trees (tree_from_list)

print("Instantiate Tree from a list")
ls_root = [5, 3, 7, 2, 4, 6, 8]
root = tree_from_list(elements=ls_root)
print(root)

Instantiate Tree from a list
5 -> (3 -> (2 -> (None, None), 4 -> (None, None)), 7 -> (6 -> (None, None), 8 -> (None, None)))


In [15]:
### Tree class on action
### Example of searching for specific values (search)

print("Searching values")
print(root.search(7))
print(root.search(14))

Searching values
7 is found in the BST
14 is not found in the BST


In [16]:
### Tree class on action
### Examples of traverse methods (inorderTraversal, preorderTraversal, postorderTraversal)

print("Original Tree")
print(root)
print("")

# In-order traverse: leftChild -> parent -> rightChild
ino = root.inorderTraversal(root)
print("In-order traversal")
print("leftChild -> parent -> rightChild")
print(ino)
print("")

# Pre-order traverse: parent -> leftChild -> rightChild
preo = root.preorderTraversal(root)
print("Pre-order traversal")
print("parent -> leftChild -> rightChild")
print(preo)
print("")

# Post-order traverse: leftChild -> rightChild -> parent
poso = root.postorderTraversal(root)
print("Post-order traversal")
print("leftChild -> rightChild -> parent")
print(poso)

Original Tree
5 -> (3 -> (2 -> (None, None), 4 -> (None, None)), 7 -> (6 -> (None, None), 8 -> (None, None)))

In-order traversal
leftChild -> parent -> rightChild
[2, 3, 4, 5, 6, 7, 8]

Pre-order traversal
parent -> leftChild -> rightChild
[5, 3, 2, 4, 7, 6, 8]

Post-order traversal
leftChild -> rightChild -> parent
[2, 4, 3, 6, 8, 7, 5]


In [17]:
### Tree class on action
### Examples of deleting nodes from Tree (delete)

print("Original Tree")
print(root)
print("")

# Delete node of the deepest level or maximum depth
root_del = root.delete(root, 6)
print("Delete node '6'")
print(root_del)
print("")

# Delete node of intermediate level
root_del = root.delete(root, 3)
print("Delete node '3'")
print(root_del)
print("")

# Delete root of Tree
root_del = root.delete(root, 5)
print("Delete node '5'")
print(root_del)

Original Tree
5 -> (3 -> (2 -> (None, None), 4 -> (None, None)), 7 -> (6 -> (None, None), 8 -> (None, None)))

Delete node '6'
5 -> (3 -> (2 -> (None, None), 4 -> (None, None)), 7 -> (None, 8 -> (None, None)))

Delete node '3'
5 -> (4 -> (2 -> (None, None), None), 7 -> (None, 8 -> (None, None)))

Delete node '5'
7 -> (4 -> (2 -> (None, None), None), 8 -> (None, None))
